In [8]:
import tensorflow as tf
import numpy as np
from selenium import webdriver
import time
import os

In [67]:
DRIVER = 'geckodriver'
driver = webdriver.Firefox()
driver.implicitly_wait(3)
driver.get('https://play2048.co')

In [55]:
def get_board(driver):
    tiles = driver.find_element_by_class_name('tile-container')
    values = [0]*16
    for tile in tiles.find_elements_by_tag_name('div'):
        className = tile.get_attribute('class')
        # lazy method of avoiding the tile-inner class
        if len(className) > 20:
            x = int(className[26]) - 1
            y = int(className[28]) - 1
            index = 4*y + x
            values[index] = int(tile.text)
    return values

In [ ]:
def get_move():
    pass

In [74]:
e = driver.find_element_by_tag_name('body')
for i in range(1000):
    values = get_board(driver)
    move = get_move(values)
    if move == 0:
        e.send_keys(u'\ue012')
    elif move == 1:
        e.send_keys(u'\ue014')
    elif move == 2:
        e.send_keys(u'\ue013')
    elif move == 3:
        e.send_keys(u'\ue015')
    time.sleep(.3)


6
7
2
6
2
6
2
2
3
6
7
2
6
7
2
6
7
2
2
10
11
10
3
6
3
7
3
3
3
3
3
3
3
3
3
3
3


KeyboardInterrupt: 